# Merge All DataFrames

In [1]:
import pandas as pd
import numpy as np

In [2]:
dir = '../data/'
prefix = 'FONNESBECK_'
suffix = '_20151202.csv'

In [51]:
adt_cms_final = pd.read_pickle(dir + './adt_cms_final.pkl')
adt_cms_final.head()

,ruid,visit_id,admit_date,discharge_date,hospital_day,stay_length,n_transfers,readmit_time,readmit_30d
0,50135262,0,2007-02-08,2007-02-12,2007-02-08,4 days,2,NaT,0
1,50135262,0,2007-02-08,2007-02-12,2007-02-09,4 days,2,NaT,0
2,50135262,0,2007-02-08,2007-02-12,2007-02-10,4 days,2,NaT,0
3,50135262,0,2007-02-08,2007-02-12,2007-02-11,4 days,2,NaT,0
4,50135262,0,2007-02-08,2007-02-12,2007-02-12,4 days,2,NaT,0


In [52]:
adt_cms_final.shape

(128389, 9)

In [53]:
phenotype = pd.read_pickle(dir + './phenotype.pkl')
phenotype.head()

,ruid,sex,dob,dod,race
0,50135262,F,1949-09-20,NaT,W
1,50135361,M,1932-02-15,2012-09-09,W
2,50135369,M,1958-05-04,2012-01-23,W
3,50135375,M,1943-05-01,2011-08-18,B
4,50135425,F,1946-10-02,NaT,W


In [54]:
merged = adt_cms_final.merge(phenotype, how='left', on='ruid')
merged.shape

(128389, 13)

In [57]:
cpt_wide = pd.read_pickle(dir + './cpt_wide.pkl')
cpt_wide.rename(columns={'event_date': 'hospital_day'}, inplace=True)
cpt_wide.head()

code_cat,ruid,hospital_day,anesthesia,eval_manage,expired,medicine,modifier,path_lab,radiology,surgery,unknown
0,50135262,2005-01-09,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,50135262,2007-02-08,0.0,1.0,2.0,4.0,2.0,10.0,4.0,0.0,0.0
2,50135262,2007-02-09,0.0,2.0,1.0,0.0,3.0,9.0,5.0,2.0,0.0
3,50135262,2007-02-10,0.0,1.0,0.0,7.0,2.0,2.0,0.0,0.0,0.0
4,50135262,2007-02-11,0.0,1.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0


In [58]:
merged = merged.merge(cpt_wide, how='left', on=['ruid', 'hospital_day'])
merged.shape

(128389, 22)

In [59]:
icd_wide = pd.read_pickle(dir + './icd_wide.pkl')
icd_wide.rename(columns={'event_date': 'hospital_day'}, inplace=True)
icd_wide.head()

code_cat,ruid,hospital_day,dx_blood,dx_circulatory,dx_congenital,dx_digestive,dx_endocrine,dx_external,dx_gu,dx_infection,...,dx_muscskel,dx_neoplasm,dx_nervous,dx_obstetric,dx_perinatal,dx_respiratory,dx_skin,dx_symptoms,proc,visit
0,50135262,2005-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50135262,2007-02-08,0.0,2.0,0.0,0.0,3.0,1.0,0.0,2.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,4.0
2,50135262,2007-02-09,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,50135262,2007-02-10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,50135262,2007-02-11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
merged = merged.merge(icd_wide, how='left', on=['ruid', 'hospital_day'])
merged.shape

(128389, 42)

In [61]:
med_classes_final_ruids = pd.read_pickle(dir + './med_classes_final_ruids.pkl')
med_classes_final_ruids.rename(columns={'entry_date': 'hospital_day'}, inplace=True)
med_classes_final_ruids.head()

,ruid,hospital_day,14-alpha Demethylase Inhibitors,5-alpha Reductase Inhibitors,"Abortifacient Agents, Nonsteroidal",Acetaldehyde Dehydrogenase Inhibitors,Acetylcholine Release Inhibitors,Acid Sensing Ion Channel Blockers,Adenosine A2 Receptor Agonists,Adenosine Deaminase Inhibitors,...,Uncoupling Agents,Uricosuric Agents,Urological Agents,Vasoconstrictor Agents,Vasodilator Agents,Viscosupplements,Vitamin B Complex,Vitamins,Voltage-Gated Sodium Channel Blockers,beta-Lactamase Inhibitors
0,50135262,2005-01-09,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,50135262,2007-01-15,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50135262,2015-01-25,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,50135262,2007-02-08,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,50135262,2007-02-09,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# select the top number of medications
counts = (med_classes_final_ruids
          .drop(columns=['ruid', 'hospital_day'])
          .fillna(0.0)
          .sum()
          .sort_values(ascending=False))

In [65]:
# keep top 10 medication class counts
keep_cols = counts[:10].index.values.tolist()
keep_cols.append('ruid')
keep_cols.append('hospital_day')
keep_cols

['Antihypertensive Agents',
 'Analgesics, Opioid',
 'Narcotics',
 'Antipyretics',
 'Anti-Bacterial Agents',
 'Anti-Inflammatory Agents, Non-Steroidal',
 'Analgesics, Non-Narcotic',
 'Antiemetics',
 'Diuretics',
 'Anti-Arrhythmia Agents',
 'ruid',
 'hospital_day']

In [66]:
top_med_classes = med_classes_final_ruids.filter(items=keep_cols)
top_med_classes.shape

(505474, 12)

In [67]:
merged = merged.merge(top_med_classes, #med_classes_final_ruids, 
                      how='left', on=['ruid', 'hospital_day'])
merged.shape

(128389, 52)

## Compress to Discharge Date Only

In [75]:
merged = (merged.drop(columns=['hospital_day'])
                .groupby(['ruid', 'visit_id', 'admit_date', 'discharge_date', 'stay_length',
                          'n_transfers', 'readmit_time', 'readmit_30d', 'sex', 'dob', 'dod', 'race'])
                .agg('sum')
                .reset_index())

In [76]:
merged.head()

,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,dob,...,Antihypertensive Agents,"Analgesics, Opioid",Narcotics,Antipyretics,Anti-Bacterial Agents,"Anti-Inflammatory Agents, Non-Steroidal","Analgesics, Non-Narcotic",Antiemetics,Diuretics,Anti-Arrhythmia Agents
0,50135361,1,2008-08-02,2008-08-05,3 days,1,218 days,0,M,1932-02-15,...,20.0,0.0,0.0,5.0,1.0,0.0,5.0,0.0,1.0,18.0
1,50135361,2,2008-10-20,2008-10-22,2 days,1,76 days,0,M,1932-02-15,...,12.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,9.0
2,50135361,3,2008-10-30,2008-11-01,2 days,1,8 days,1,M,1932-02-15,...,11.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,7.0
3,50135361,4,2008-11-04,2008-11-08,4 days,2,3 days,1,M,1932-02-15,...,9.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,3.0,8.0
4,50135361,5,2008-12-02,2008-12-05,3 days,3,24 days,1,M,1932-02-15,...,9.0,0.0,0.0,4.0,2.0,4.0,0.0,0.0,6.0,11.0


In [77]:
merged.tail()

,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,dob,...,Antihypertensive Agents,"Analgesics, Opioid",Narcotics,Antipyretics,Anti-Bacterial Agents,"Anti-Inflammatory Agents, Non-Steroidal","Analgesics, Non-Narcotic",Antiemetics,Diuretics,Anti-Arrhythmia Agents
5025,53736394,7,2008-12-24,2008-12-29,5 days,2,7 days,1,M,1927-03-18,...,13.0,9.0,7.0,11.0,11.0,9.0,11.0,3.0,13.0,8.0
5026,53736394,8,2009-01-03,2009-01-08,5 days,2,5 days,1,M,1927-03-18,...,1.0,3.0,2.0,9.0,24.0,3.0,9.0,7.0,1.0,0.0
5027,53736398,1,2005-04-09,2005-04-12,3 days,0,18 days,1,M,1937-09-17,...,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
5028,53736398,2,2005-04-12,2005-04-20,8 days,1,0 days,1,M,1937-09-17,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
5029,53736398,3,2005-04-20,2005-04-22,2 days,1,0 days,1,M,1937-09-17,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [78]:
merged.to_csv(dir + './merged.csv')